# vLLM, llama.cpp и SGLang

## Зачем  нужны движки инференса?

* Обучение LLM — **дорого и редко**
* Инференс — **часто, массово и в продакшене**
* Наивный PyTorch-инференс:

  * медленный
  * плохо масштабируется
  * неэффективно использует GPU / CPU

**Движки инференса решают:**

* высокую пропускную способность (throughput)
* низкую задержку (latency)
* экономию памяти


## Общая картина

| Движок        | Где используется           |
| ------------- | -------------------------- |
| **vLLM**      | Серверы, API, облака       |
| **llama.cpp** | Локальные машины, embedded |
| **SGLang**    | Агентные системы, RAG      |


# vLLM

**vLLM** — это высокопроизводительный движок инференса LLM на GPU.

Основная идея: **максимально эффективно использовать GPU-память и батчинг запросов**

OpenAI-compatible API


Важная идея vLLM — PagedAttention

Проблема:

* KV-cache занимает много памяти
* разные запросы имеют разную длину

Решение:

* KV-cache хранится как **страницы (pages)**
* динамическое выделение и переиспользование

Плюсы:

* меньше fragmentation
* больше одновременных запросов


## Пример использования vLLM

```python
from vllm import LLM, SamplingParams

llm = LLM(model="meta-llama/Llama-2-7b-chat-hf")
params = SamplingParams(temperature=0.8, max_tokens=128)

outputs = llm.generate(["Explain transformers"], params)
print(outputs[0].outputs[0].text)
```

- Нужно обслуживать **много пользователей**
- Есть **GPU**
- Важен throughput


# llama.cpp


**llama.cpp** — C++ реализация инференса LLaMA-подобных моделей.

Цели:

* минимальные зависимости
* работа на CPU
* переносимость

Запускается:

* на ноутбуке
* на Raspberry Pi
* на сервере без GPU

Проблема:

* FP16/FP32 слишком большие

Решение:

* INT8 / INT4 / INT2
* форматы GGML / GGUF

Пример:

* 7B модель

  * FP16 ≈ 14 GB
  * Q4 ≈ 4 GB


## Архитектура llama.cpp

* Чистый C/C++
* SIMD (AVX, NEON)
* OpenMP
* Metal / CUDA (опционально)

Фокус:

* latency одного запроса
* энергоэффективность


## Пример запуска llama.cpp

```bash
./llama-cli \
  -m llama-7b-q4.gguf \
  -p "Explain backpropagation"
```

Или через Python bindings:

```python
from llama_cpp import Llama
llm = Llama(model_path="model.gguf")
llm("Hello")
```

* Локальный запуск
* Нет GPU


# SGLang


**SGLang** — runtime для управления LLM-инференсом.

* control-flow
* программирование диалогов
* агентные сценарии


Наивный код:

* много prompt engineering
* сложная логика в Python

SGLang:

* декларативные шаблоны
* разделение логики и инференса


## Пример SGLang

```python
@sgl.function
def qa(ctx, question):
    ctx += "Answer the question:"
    ctx += question
    ctx += sgl.gen("answer", max_tokens=100)
    return ctx["answer"]
```

## Архитектура SGLang

* Frontend: DSL / Python API
* Runtime: scheduler
* Backend:

  * vLLM
  * TensorRT-LLM

SGLang **использует** vLLM, но не заменяет его

## Где используется SGLang

* RAG pipelines
* multi-step reasoning
* tool calling
* агенты

Особенно полезен когда:

* сложная логика генерации
* много взаимодействий с LLM

## Пример с `llama.cpp` и `OpenAI API`

Запуск:

`docker run -p 8080:8080 ghcr.io/ggml-org/llama.cpp:server -hf ggml-org/gemma-3-270m-it-GGUF --port 8080 --host 0.0.0.0`

In [4]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:8080/v1", 
    api_key="sk-no-key-required" 
)

completion = client.chat.completions.create(
    model="local-model", 
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Explain the concept of RAG"}
    ],
)

print(completion.choices[0].message.content)

Okay, I'm ready to explain the concept of RAG (Retrieval-Augmented Generation). Let's break it down!

**What is RAG?**

RAG is a technique that combines the strengths of traditional search engines (like Google) with the power of generative AI. It aims to provide more relevant and accurate information to users in the future, especially for tasks that require factual knowledge.

Think of it like this:

*   **Traditional Search:** You type a keyword into a search box. The search engine looks for relevant results based on your keywords.
*   **RAG:** You type a keyword into a search box and then ask the search engine to retrieve relevant information from a vast database of websites and information.

**Key Components of RAG:**

1.  **Knowledge Base:** This is the core of the system. It's a collection of websites, articles, and other resources that the system can access to answer questions and provide answers. This could be a library of books, a website with a comprehensive FAQ, or even a dat